Verifying the files we have in the databricks file system

In [2]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/aisles.csv,aisles.csv,2603
dbfs:/FileStore/tables/departments.csv,departments.csv,270
dbfs:/FileStore/tables/order_products__prior.csv,order_products__prior.csv,577550706
dbfs:/FileStore/tables/order_products__train.csv,order_products__train.csv,24680147
dbfs:/FileStore/tables/orders.csv,orders.csv,108968645
dbfs:/FileStore/tables/products.csv,products.csv,2166953
dbfs:/FileStore/tables/sample_submission.csv,sample_submission.csv,1475693


Importing all the available files into the spark dataframe and creating temporary tables

In [4]:
aisles = spark.read.csv("/FileStore/tables/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("/FileStore/tables/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv("/FileStore/tables/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/FileStore/tables/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/FileStore/tables/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/FileStore/tables/products.csv", header=True, inferSchema=True)

aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

Let’s take a look at the top 5 rows of each of the imported file.

Top 5 orders in the orders dataframe

In [6]:
orders.show(n=5)

+--------+-------+--------+------------+---------+-----------------+----------------------+
order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
 2539329| 1| prior| 1| 2| 8| null|
 2398795| 1| prior| 2| 3| 7| 15.0|
 473747| 1| prior| 3| 3| 12| 21.0|
 2254736| 1| prior| 4| 4| 7| 29.0|
 431534| 1| prior| 5| 4| 15| 28.0|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 5 rows

Top 5 orders in the products dataframe

In [8]:
products.show(n=5)

+----------+--------------------+--------+-------------+
product_id| product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
 1|Chocolate Sandwic...| 61| 19|
 2| All-Seasons Salt| 104| 13|
 3|Robust Golden Uns...| 94| 7|
 4|Smart Ones Classi...| 38| 1|
 5|Green Chile Anyti...| 5| 13|
+----------+--------------------+--------+-------------+
only showing top 5 rows

Top 5 orders in the order_products_train dataframe

In [10]:
order_products_train.show(n=5)

+--------+----------+-----------------+---------+
order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
 1| 49302| 1| 1|
 1| 11109| 2| 1|
 1| 10246| 3| 0|
 1| 49683| 4| 0|
 1| 43633| 5| 1|
+--------+----------+-----------------+---------+
only showing top 5 rows

Top 5 orders in the order_products_prior dataframe

In [12]:
order_products_prior.show(n=5)

+--------+----------+-----------------+---------+
order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
 2| 33120| 1| 1|
 2| 28985| 2| 1|
 2| 9327| 3| 0|
 2| 45918| 4| 1|
 2| 30035| 5| 0|
+--------+----------+-----------------+---------+
only showing top 5 rows

Top 5 orders in the departments dataframe

In [14]:
departments.show(n=5)

+-------------+----------+
department_id|department|
+-------------+----------+
 1| frozen|
 2| other|
 3| bakery|
 4| produce|
 5| alcohol|
+-------------+----------+
only showing top 5 rows

Top 5 orders in the aisles dataframe

In [16]:
aisles.show(n=5)

+--------+--------------------+
aisle_id| aisle|
+--------+--------------------+
 1|prepared soups sa...|
 2| specialty cheeses|
 3| energy granola bars|
 4| instant foods|
 5|marinades meat pr...|
+--------+--------------------+
only showing top 5 rows

EXPLORATORY DATA ANALYSIS

On which day of the week customers purchase the most?

▶ Sunday and Monday have the most orders, while Thursday has the least orders in a week

In [18]:
%sql
select count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from orders  
 group by order_dow 
 order by total_orders desc

total_orders,day_of_week
600905,Sunday
587478,Monday
467260,Tuesday
453368,Friday
448761,Saturday
436972,Wednesday
426339,Thursday


What time of day do customers purchase?

▶ Below line graph shows that the customers are more likely to place an order between 9 am to 6 pm

In [20]:
%sql
select count(order_id) as total_orders, order_hour_of_day as hour 
from orders 
group by order_hour_of_day 
order by order_hour_of_day

total_orders,hour
22758,0
12398,1
7539,2
5474,3
5527,4
9569,5
30529,6
91868,7
178201,8
257812,9


How often do customers place orders?

▶ It appears that most of the customers order once a week since the majority of records are concentrated between 0 to 7 days
▶ Also, a large number of customer place their order 30 days or later days since because ‘days_since_prior’ column is capped at 30

In [22]:
%sql
select days_since_prior_order,count(order_id) as total_orders
from orders 
group by days_since_prior_order 
order by days_since_prior_order

days_since_prior_order,total_orders
null,206209
0.0,67755
1.0,145247
2.0,193206
3.0,217005
4.0,221696
5.0,214503
6.0,240013
7.0,320608
8.0,181717


Let’s create a Master Dataset by merging together products, departments, order_products_train, and order_products_prior datasets together and run the query on top of that.

In [24]:
%sql
create table master_table as
(select op.*,p.product_name,p.aisle_id,p.department_id,d.department from
 (select * from order_products_train 
 union
 select * from order_products_prior) as op
 inner join products as p
 on op.product_id = p.product_id
 inner join departments as d
 on p.department_id = d.department_id)

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table default.master_table already exists. You need to drop it first.;
	at org.apache.spark.sql.execution.command.CreateDataSourceTableAsSelectCommand.run(createDataSourceTables.scala:158)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:119)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:206)
	at org.apache.spark.sql.Dataset$$anonfun$6.apply(Dataset.scala:206)
	at org.apache.spark.sql.Dataset$$anonfun$54.apply(Dataset.scala:3492)
	at org.apache.spark.sql.Dataset$$anonfun$54.apply(Dataset.scala:3487)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:113)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:242)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:99)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:172)
	at org.apache.spark.sql.Dataset.org$apache$spark$sql$Dataset$$withAction(Dataset.scala:3487)
	at org.apache.spark.sql.Dataset. (Dataset.scala:206)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:90)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:696)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:716)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:88)
	at com.databricks.backend.daemon.driver.SQLDriverLocal$$anonfun$1.apply(SQLDriverLocal.scala:34)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.immutable.List.map(List.scala:296)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:34)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:141)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:396)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:373)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:49)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:49)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:373)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at com.databricks.backend.daemon.driver.DriverWrapper$$anonfun$tryExecutingCommand$2.apply(DriverWrapper.scala:644)
	at scala.util.Try$.apply(Try.scala:192)
	at com.databricks.backend.daemon.driver.DriverWrapper.tryExecutingCommand(DriverWrapper.scala:639)
	at com.databricks.backend.daemon.driver.DriverWrapper.getCommandOutputAndError(DriverWrapper.scala:485)
	at com.databricks.backend.daemon.driver.DriverWrapper.executeCommand(DriverWrapper.scala:597)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInnerLoop(DriverWrapper.scala:390)
	at com.databricks.backend.daemon.driver.DriverWrapper.runInner(DriverWrapper.scala:337)
	at com.databricks.backend.daemon.driver.

How many items do customers purchase in an order?

▶ The below bar chart depicts that the most common number of items purchased in order by customers is 4
▶ Majority of customers prefer to purchase between 1 to 15 items per order

In [26]:
%sql
select order_id,count(product_id) as total_items
from master_table 
group by order_id 

order_id,total_items
1591,31
10362,36
372179,20
401778,15
439283,28
503098,17
699436,10
720673,12
725345,28
760742,19


Which are the top departments from which orders are placed?

▶ If we take a look at top 10 departments from which most items are purchased, we would infer that almost 50% of the items purchased belong from just 2 departments which are ‘produce’ and ‘dairy eggs’

In [28]:
%sql
select department, count(*) as orders_count from master_table
group by department
order by orders_count desc
limit 10

department,orders_count
produce,9888378
dairy eggs,5631067
snacks,3006412
beverages,2804175
frozen,2336858
pantry,1956819
bakery,1225181
canned goods,1114857
deli,1095540
dry goods pasta,905340


Which are the most purchased items?

▶ These are the top 8 items bought by Instacart customers in their orders. Banana seems to be most bought commonly bought item in baskets followed by strawberries, baby spinach, avocado, etc.

In [30]:
%sql
select product_name, count(*) as orders_count from master_table
group by product_name
order by orders_count desc
limit 200

product_name,orders_count
Banana,491291
Bag of Organic Bananas,394930
Organic Strawberries,275577
Organic Baby Spinach,251705
Organic Hass Avocado,220877
Organic Avocado,184224
Large Lemon,160792
Strawberries,149445
Limes,146660
Organic Whole Milk,142813


Let’s also make a word cloud of the top 200 items bought by Instacart customers.

In [32]:
# !pip install wordcloud
from wordcloud import WordCloud 
import matplotlib.pyplot as plt

In [33]:
df = sqlContext.sql("SELECT product_name FROM (select product_name, count(*) as orders_count from master_table group by product_name order by orders_count desc limit 200)")
df2 = df.rdd.flatMap(lambda x: x).collect()
fullStr = ' '.join(df2)

In [34]:
wordcloud = WordCloud(background_color="white").generate(fullStr)

# Display the generated image:
plt.figure(figsize=(14, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
display()

From the word cloud, it appears that Americans buy organic food and veggies a lot as words like Organic, Milk, Water, Apple, Sparling, Egg, Green, Cheese, etc. are getting highlighted the most.

FP-Growth Algorithm

Organizing the data by the shopping basket:

For implementing FP-growth, first, we would be creating baskets of each order in our dataset. We would do so by creating a baskets data frame having 2 columns: first, the order_id and second, the list of items bought in that particular order.

In [37]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')
rawData.show(5)
baskets.show(5)
display(baskets)

order_id,items
1342,"List(Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas)"
1591,"List(Cracked Wheat, Strawberry Rhubarb Yoghurt, Organic Bunny Fruit Snacks Berry Patch, Goodness Grapeness Organic Juice Drink, Honey Graham Snacks, Spinach, Granny Smith Apples, Oven Roasted Turkey Breast, Pure Vanilla Extract, Chewy 25% Low Sugar Chocolate Chip Granola, Banana, Original Turkey Burgers Smoke Flavor Added, Twisted Tropical Tango Organic Juice Drink, Navel Oranges, Lower Sugar Instant Oatmeal Variety, Ultra Thin Sliced Provolone Cheese, Natural Vanilla Ice Cream, Cinnamon Multigrain Cereal, Garlic, Goldfish Pretzel Baked Snack Crackers, Original Whole Grain Chips, Medium Scarlet Raspberries, Lemon Yogurt, Original Patties (100965) 12 Oz Breakfast, Nutty Bars, Strawberry Banana Smoothie, Green Machine Juice Smoothie, Coconut Dreams Cookies, Buttermilk Waffles, Uncured Genoa Salami, Organic Greek Whole Milk Blended Vanilla Bean Yogurt)"
4519,List(Beet Apple Carrot Lemon Ginger Organic Cold Pressed Juice Beverage)
4935,List(Vodka)
6357,"List(Globe Eggplant, Panko Bread Crumbs, Fresh Mozzarella Ball, Grated Parmesan, Gala Apples, Italian Pasta Sauce Basilico Tomato, Basil & Garlic, Organic Basil, Banana, Provolone)"
10362,"List(Organic Baby Spinach, Organic Spring Mix, Organic Leek, Slow Roasted Lightly Seasoned Chick'n, Organic Basil, Organic Shredded Mild Cheddar, Bag of Organic Bananas, Sliced Baby Bella Mushrooms, Organic Tapioca Flour, Organic Gala Apples, Lemons, Limes, Pitted Dates, Jalapeno Peppers, Original Tofurky Deli Slices, Organic Red Bell Pepper, Organic Shredded Carrots, Roma Tomato, Crinkle Cut French Fries, Large Greenhouse Tomato, Organic Pinto Beans, Organic Three Grain Tempeh, Organic Garnet Sweet Potato (Yam), Organic Coconut Milk, Organic Extra Firm Tofu, Ground Sausage Style Veggie Protein, Extra Virgin Olive Oil, Hass Avocados, Multigrain Tortilla Chips, The Ultimate Beefless Burger, Yellow Bell Pepper, Coconut Flour, Light Brown Sugar, Organic Harissa Seasoning, Crushed Garlic, Organic Whole Cashews)"
19204,"List(Reduced Fat Crackers, Dishwasher Cleaner, Peanut Powder, Disinfecting Wipes Lemon & Fresh Scent, Lemon Lime Thirst Quencher, Fat Free & Lower Sodium Chicken Broth, American Blend Salad, Cinnamon Cereal, Extra Nasal Strips, Reduced Fat Creamy Peanut Butter Spread, Mozzarella String Cheese Sticks, Light Low-Moisture Part Skim, Electrolyte Enhanced Water, Original Petroleum Jelly, High Efficiency Complete Dual Formula)"
29601,"List(Organic Red Onion, Small Batch Authentic Taqueria Tortilla Chips, Hummus, Hope, Original Recipe, Unsweetened Whole Milk Peach Greek Yogurt, Toasted Coconut Almondmilk Blend, Skillet Refried Red Beans Sautéed With Onion & Tomatillo, Almondmilk, Pure, Chocolate Protein, Organic Greek Lowfat Yogurt With Strawberries, Bag of Organic Bananas, California Orange Juice, Mini Whole Wheat Pita Bread, Coconut Almond Creamer Blend, Banana Chia Pod, Tomatillo Salsa, SALSA FRNTR CHPTL SALSA, Guacamole, Water)"
31035,"List(Organic Cripps Pink Apples, Organic Golden Delicious Apple, Organic Navel Orange, Bag of Organic Bananas)"
40011,"List(Organic Baby Spinach, Organic Blues Bread with Blue Cornmeal Crust, Sea Salt Macadamias, Natural Calm Magnesium Supplement Raspberry Lemon Flavor Powder, Chocolate Coconut Protein Bar, Sport Chocolate Mint Protein Bar)"


Above are the top 5 rows of the baskets data frame, to be fed into the FP-growth algorithm.

Implementation of FP-growth algorithm using Scala:

Here, we would be using spark.ml’s FP-growth package for implementation.

In [39]:
%scala
import org.apache.spark.ml.fpm.FPGrowth

// Extract out the items 
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Use FPGrowth
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

// Display frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_056f6cf2dad5
model: org.apache.spark.ml.fpm.FPGrowthModel = fpgrowth_056f6cf2dad5
mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]
ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 2 more fields]

Now, let us explore the most frequent basket of items (containing at least 2 items).

In [41]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",710
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",649
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",587
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",531
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",497
"List(Organic Avocado, Organic Baby Spinach, Banana)",484
"List(Organic Avocado, Large Lemon, Banana)",477
"List(Limes, Large Lemon, Banana)",452
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",424
"List(Limes, Organic Avocado, Large Lemon)",389


The most frequent basket of items comprises of organic avocado, organic strawberries, and organic bananas together.

A good way to think about association rules is that model determines that if you purchased something (i.e. the antecedent), then you will purchase this other thing (i.e. the consequent) with the following confidence.

In [43]:
%sql
select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5984251968503937
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.546875
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5459770114942529
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5412186379928315
"List(Yellow Onions, Strawberries)",List(Banana),0.5357142857142857
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5314685314685315
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.521099116781158
"List(Organic D'Anjou Pears, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5170454545454546
"List(Organic Unsweetened Almond Milk, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5141065830721003


If a customer has organic raspberries, organic avocados, and organic strawberries in its basket, then it may make sense to recommend organic bananas as well. Surprisingly, the top 10 purchase recommendations either organic bananas or bananas.

Implementation of FP-growth algorithm — Market basket analysis using PySpark:

In [45]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0)
model = fpGrowth.fit(baskets)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the consequents as prediction
model.transform(baskets).show()

+--------------------+-----+
 items| freq|
+--------------------+-----+
[Organic Tomato B...| 772|
[Organic Tomato B...| 175|
[Organic Tomato B...| 144|
[Organic Tomato B...| 179|
[Organic Spinach ...| 475|
[Whole Milk Ricot...| 347|
 [Medium Salsa Roja]| 275|
 [Ground Buffalo]| 231|
 [Tonic Water]| 194|
[Original Coconut...| 173|
[Low-Fat Strawber...| 152|
[Organic SprouTof...| 137|
 [Banana]|18726|
[Fruit Punch Spor...| 275|
[Kitchen Cheese E...| 230|
[Country White Br...| 194|
[Soft & Smooth Wh...| 173|
[Natural Liquid L...| 152|
[Bag of Organic B...|15480|
[Organic Large Gr...| 769|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+
 antecedent| consequent| confidence| lift|
+--------------------+--------------------+-------------------+------------------+
[Broccoli Crown, ...| [Banana]| 0.3690773067331671|2.5860442347085395|
 [Sugar Snap Peas]|[Bag of Organic B...| 0.2207001522070015|1.8706619038067482|
[Organic Red Onio...|[Bag of Organic B...|0.34673366834170855|2.9389262202485296|
[Organic Red Onio...|[Organic Baby Spi...|0.22780569514237856|3.0550038280801664|
[85% Lean Ground ...| [Banana]| 0.235202492211838|1.6480125921511828|
[Organic Small Bu...|[Bag of Organic B...|0.36633663366336633| 3.105081612812444|
[Organic Zucchini...| [Large Lemon]|0.34615384615384615| 5.583097725875844|
[Organic Zucchini...|[Organic Baby Spi...|0.29554655870445345|3.9634473038688296|
[Organic Zucchini...| [Limes]|0.15368421052631578| 3.342408682073159|
[Organic Zucchini...| [Organic Avocado]|0.16210526315789472|2.8707881594930775|
[Organic Zucchini...| [Large Lemon]| 0.1957894736842105| 3.157878433021706|
[Organic Zucchini...|[Organic Strawber...| 0.2031578947368421|2.4468647154880037|
[Organic Zucchini...|[Bag of Organic B...| 0.2694736842105263|2.2840679994560045|
[Organic Zucchini...| [Banana]| 0.2326315789473684| 1.629998763329342|
[Organic Zucchini...| [Organic Cucumber]|0.15157894736842106| 4.311407349937819|
[Organic Zucchini...|[Organic Hass Avo...|0.17473684210526316| 3.14370578853551|
[Organic Zucchini...|[Organic Yellow O...| 0.1463157894736842| 4.47504625199362|
[Organic Zucchini...| [Organic Garlic]| 0.1463157894736842| 4.61711121237437|
[Sparkling Water ...|[Lime Sparkling W...|0.18627450980392157|12.431786448048191|
[Sparkling Water ...|[Sparkling Water ...|0.28431372549019607| 11.10584090736622|
+--------------------+--------------------+-------------------+------------------+
only showing top 20 rows

+--------+--------------------+--------------------+
order_id| items| prediction|
+--------+--------------------+--------------------+
 1342|[Raw Shrimp, Seed...|[Organic Tomato B...|
 1591|[Cracked Wheat, S...|[Bag of Organic B...|
 4519|[Beet Apple Carro...| []|
 4935| [Vodka]| []|
 6357|[Globe Eggplant, ...|[Organic Tomato B...|
 10362|[Organic Baby Spi...|[Organic Strawber...|
 19204|[Reduced Fat Crac...|[Bag of Organic B...|
 29601|[Organic Red Onio...|[Organic Tomato B...|
 31035|[Organic Cripps P...|[Organic Tomato B...|
 40011|[Organic Baby Spi...|[Organic Tomato B...|
 46266|[Uncured Beef Hot...|[Organic Tomato B...|
 51607|[Donut House Choc...| []|
 58797|[Concentrated But...|[Organic Tomato B...|
 61793|[Raspberries, Gre...|[Hass Avocados, S...|
 67089|[Original Tofurky...|[Organic Tomato B...|
 70863|[Extra Hold Non-A...|[Limes, Organic W...|
 88674|[Organic Coconut ...|[Organic Cilantro...|
 91937|[No. 485 Gin, Mon...| []|
 92317|[Red Vine Tomato,...|[Bag of Organic B...|
 99621|[Organic Baby Aru...|[Limes, Organic W...|
+--------+--------------------+--------------------+
only showing top 20 rows